In [27]:
import requests
import datetime
import pandas as pd
import time
import zipfile
import warnings
from tqdm.notebook import tqdm
import io

# 警告を特定のものに限定する
warnings.filterwarnings('ignore', category=DeprecationWarning)

class GetDocid:
    # 1 コンストラクタ・日付リストの作成
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self.day_list = self.create_day_list()

    def create_day_list(self):
        day_list = []
        period = (self.end_date - self.start_date).days
        for d in range(period + 1):
            day = self.start_date + datetime.timedelta(days=d)
            day_list.append(day)
        return day_list

    # 2 レポートリストの作成
    def create_report_list(self):
        report_list = []
        for day in self.day_list:
            url = "https://api.edinet-fsa.go.jp/api/v2/documents.json"
            params = {"date": day, "type": 2, "Subscription-Key": "発行したAPI鍵"}
            try:
                res = requests.get(url, params=params)
                res.raise_for_status()
                json_data = res.json()
                time.sleep(3)  # APIのレート制限に従って調整
            except requests.RequestException as e:
                print(f"Request failed: {e}")
                continue

            for result in json_data.get("results", []):
                if result["ordinanceCode"] == "010" and result["formCode"] == "030000":
                    report_list.append({
                        '会社名': result["filerName"],
                        '書類名': result["docDescription"],
                        'docID': result["docID"],
                        '証券コード': result["secCode"],
                        'ＥＤＩＮＥＴコード': result["edinetCode"],
                        '決算期': result["periodEnd"],
                        '提出日': day
                    })
        return report_list

    # 3 データフレームの作成と保存
    def create_docid_df(self, base_dir):
        # ファイルパスを動的に設定
        zip_path = "/content/Edinetcode.zip"
        extract_dir = "/content/Edinetcode"
        df_info_path = f"{extract_dir}/EdinetcodeDlInfo.csv"
        output_path = f"{base_dir}/edinet_df.csv"

        #edinetcodelistを取得する
        !wget https://disclosure2dl.edinet-fsa.go.jp/searchdocument/codelist/Edinetcode.zip

        # zipファイルのダウンロードと展開
        try:
            with zipfile.ZipFile(zip_path) as zip_f:
                zip_f.extractall("Edinetcode")
        except zipfile.BadZipFile:
            print("Failed to unzip file")
            return None

        df_info = pd.read_csv(df_info_path, encoding="cp932", skiprows=[0])
        df_report = pd.DataFrame(self.create_report_list())
        df_info = df_info[["ＥＤＩＮＥＴコード", "提出者業種"]]
        merged_df = pd.merge(df_report, df_info, how="inner", on="ＥＤＩＮＥＴコード")
        merged_df.to_csv(output_path,index=False)

        return merged_df


In [28]:
# 書類提出期間の指定
start_date = datetime.date(2024, 3,1)
end_date = datetime.date(2024,3,15)

# データ保存のためのディレクトリの指定
base_dir = "/content/"

In [29]:
gd = GetDocid(start_date,end_date)

In [31]:
%%time
edinet_df = gd.create_docid_df(base_dir)

--2025-05-26 01:16:15--  https://disclosure2dl.edinet-fsa.go.jp/searchdocument/codelist/Edinetcode.zip
Resolving disclosure2dl.edinet-fsa.go.jp (disclosure2dl.edinet-fsa.go.jp)... 13.107.246.40, 2620:1ec:bdf::40
Connecting to disclosure2dl.edinet-fsa.go.jp (disclosure2dl.edinet-fsa.go.jp)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 554532 (542K) [application/octet-stream]
Saving to: ‘Edinetcode.zip.2’

Edinetcode.zip.2    100%[===================>] 541.54K  1.14MB/s    in 0.5s    

2025-05-26 01:16:17 (1.14 MB/s) - ‘Edinetcode.zip.2’ saved [554532/554532]

CPU times: user 545 ms, sys: 73.3 ms, total: 619 ms
Wall time: 1min


In [32]:
edinet_df

,会社名,書類名,docID,証券コード,ＥＤＩＮＥＴコード,決算期,提出日,提出者業種
0,ローランド株式会社,有価証券報告書－第52期(2023/01/01－2023/12/31),S100SZWZ,79440,E01834,2023-12-31,2024-03-05,その他製品
1,カゴメ株式会社,有価証券報告書－第80期(2023/01/01－2023/12/31),S100T0O1,28110,E00444,2023-12-31,2024-03-08,食料品
2,株式会社ＫＧ情報,有価証券報告書－第44期(2022/12/21－2023/12/20),S100T0Z8,24080,E05416,2023-12-20,2024-03-11,サービス業
3,協和キリン株式会社,有価証券報告書－第101期(2023/01/01－2023/12/31),S100T1DP,41510,E00816,2023-12-31,2024-03-12,医薬品
4,Ｂ－Ｒ サーティワン アイスクリーム株式会社,有価証券報告書－第51期(2023/01/01－2023/12/31),S100T1DX,22680,E00474,2023-12-31,2024-03-14,食料品
5,太洋テクノレックス株式会社,有価証券報告書－第63期(2022/12/21－2023/12/20),S100T1ZT,66630,E02097,2023-12-20,2024-03-15,電気機器
6,株式会社Ｖｅｒｉｔａｓ Ｉｎ Ｓｉｌｉｃｏ,有価証券報告書－第8期(2023/01/01－2023/12/31),S100T209,130A0,E39268,2023-12-31,2024-03-15,医薬品


In [33]:
docid_list = edinet_df["docID"].tolist()

In [37]:
class GetPDFFromEdinet:

    def __init__(self,docid_list):
        self.docid_list = docid_list

    def get_pdf_file(self):
        for docid in tqdm(self.docid_list):
            # 書類取得APIのエンドポイント
            url = "https://api.edinet-fsa.go.jp/api/v2/documents/" + docid
            print(url)
            time.sleep(5)


            #書類取得APIのパラメータ
            params ={"type":2,"Subscription-Key": "発行したAPI鍵"}
            res = requests.get(url,params=params, verify=False)

            filename = docid + ".pdf"

            #ファイルへの出力
            print(res.status_code)
            if res.status_code == 200:
                with open(filename, 'wb') as f:
                    for chunk in res.iter_content(chunk_size=1024):
                        f.write(chunk)

In [38]:
gpf = GetPDFFromEdinet(docid_list)

In [39]:
gpf.get_pdf_file()

  0%|          | 0/7 [00:00<?, ?it/s]

https://api.edinet-fsa.go.jp/api/v2/documents/S100SZWZ


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
https://api.edinet-fsa.go.jp/api/v2/documents/S100T0O1


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

200
https://api.edinet-fsa.go.jp/api/v2/documents/S100T1ZT


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
https://api.edinet-fsa.go.jp/api/v2/documents/S100T209


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
